# Working with Distinct Values, $elemMatch, and Regex

## An exceptional laureate

In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = client.nobel

In [ ]:
db.laureates.find_one({"prizes.2": {"$exists": True}})

## Using .distinct()

In [ ]:
db.laureates.distinct("gender")

- A convenience method for a common aggregation (like `count_documents`)
- We will not cover custom aggregations in this lesson, but the `aggregate` method is powerful.
- `distinct` aggregation is efficient if there is a collection *index* on the field
- We will learn how to create an index later in this lesson
- No index needed here: collection fits in memory, has ≲ 1,000 documents

## All prize categories vs those with multi-winners

In [ ]:
db.laureates.distinct("prizes.category")

In [ ]:
db.laureates.distinct(
  "prizes.category", {"prizes.1": {"$exists": True}})

## Distinct mini-exercise: Born here, went there

In which countries have USA-born laureates had affiliations for their prizes?

In [ ]:
set(db.laureates.distinct("prizes.affiliations.country",{"bornCountry":"USA"}))

## Enter $elemMatch

In [ ]:
db.laureates.count_documents({
    "prizes": {
        "category": "physics",
        "share": "1"
    }
})

Why is the above number zero?

In [ ]:
db.laureates.count_documents({
    "prizes.category": "physics",
    "prizes.share": "1"
})

In [ ]:
db.laureates.count_documents({
    "prizes": {
        "$elemMatch": {
            "category": "physics",
            "share": "1"
        }
    }
})

Why are these numbers different?

In [ ]:
db.laureates.find_one({"firstname": "Marie"})

## $elemMatch mini-exercise: Sharing in physics after World War II

What is the approximate ratio of the number of laureates who won an *unshared*, i.e.

    {"share": "1"}

, prize in physics after World War II, i.e.

    {"year": {"$gte": "1945"}}

, to the number of laureates who won a *shared* prize in physics after World War II?

In [ ]:
# Save a filter for laureates with unshared prizes
unshared = {
    "prizes": {"$elemMatch": {
        "category": "physics",
        "share": "1",
        "year": {"$gte": "1945"},
    }}}

# Save a filter for laureates with shared prizes
shared = {
    "prizes": {"$elemMatch": {
        "category": "physics",
        "share": {"$gt": "1"},
        "year": {"$gte": "1945"},
    }}}

ratio = db.laureates.count_documents(unshared) / db.laureates.count_documents(shared)
print(ratio)

## Finding a substring with $regex

In [ ]:
case_sensitive = db.laureates.distinct(
    "bornCountry",
    {"bornCountry": {"$regex": "Poland"}})
print(case_sensitive)

## Beginning and ending

In [ ]:
from bson.regex import Regex

db.laureates.distinct("bornCountry", 
                      {"bornCountry": Regex("^Poland")})

In [ ]:
db.laureates.distinct(
    "bornCountry", 
     {"bornCountry": Regex("^Poland \(now")})

In [ ]:
db.laureates.distinct(
    "bornCountry", 
     {"bornCountry": Regex("now Poland\)$")})

## Regex mini-exercise: Glenn, George, and others in the G.S. crew

There are two laureates with Berkeley, California as a prize affiliation city that have the initials G.S. - Glenn Seaborg and George Smoot.

How many laureates in total have a first name beginning with "G" and a surname beginning with "S"?

In [ ]:
db.laureates.count_documents({"firstname": Regex("^G"), "surname": Regex("^S")})